In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Fazit
Abschließend soll ein Fazit zum entwickelten Mühle-Spiel gezogen werden. Dazu wird die Performance der implementierten Algorithmen Alpha-Beta und Minimax verglichen und beurteilt. Des Weiteren soll analysiert werden, mit welchen Maßnahmen die Künstliche Intelligenz weiter verbessert werden könnte.

## Alpha-Beta vs. Minimax
Für die folgende Auswertung ist zu beachten, dass die Spiele auf einem Rechner mit folgender Spezifikation durchgeführt wurden:
* Prozessor: AMD Ryzen 5 3600x
* RAM: 16Gb DDR4

Zur Bewertung der zeitlichen Performance wurden Spiele durchgeführt, bei denen die beiden Algorithmen Alpha-Beta-Pruning und Minimax mit der gleichen Suchtiefe gegeneinander gespielt haben. Die Ergebnisse sind der unten aufgeführten Tabelle sowie den Grafiken zu entnehmen.

Suchtiefe | Ø Zugzeit Alpha-Beta (s) | Ø Zugzeit Minimax (s) | MM/AB
:--:|--|--|--
1 | 0,,000867  | 0,003282 | 378,34%
2 | 0,037437 | 0,024096 | 64,36%
3 | 0,251854  | 0,373193 | 148,17%
4 | 0,943550 | 2,064215 | 218,77%


<img src="images\linear.png" alt="Durchschnittliche Zugzeit, lineare Achse" width="300"/>
<img src="images\logarithmic.png" alt="Durchschnittliche Zugzeit, logarithmische Achse" width="300"/>

Zu sehen ist, dass sowohl der absolute als auch der relative Abstand mit zunehmender Suchtiefe exponentiell steigen. Das heißt, dass gerade bei einer großen Suchtiefe Alpha-Beta-Pruning wesentlich schneller ist. Doch eine höhere Suchtiefe führt zu einem besseren Spiel der künstlichen Intelligenz und somit zu einer höheren Gewinnwahrscheinlichkeit. Deshalb ist für diesen Anwendungsfall als Such-Algorithmus Alpha-Beta-Pruning die bessere Wahl.




## Automatisierte Auswertung

Die folgende Funktion `restart_auto_play` besitzt die gleiche Funktion wie `restart(b)` aus den Notebook Muehle_GUI, welche dort durch den Neustart-Knopf aufgerufen wird. Hier wird `restart_auto_play` aufgerufen, um das Spiel für mehrere Druchläufe zur Auswertung neu zu starten.

In [ ]:
def restart_auto_play(): 
    gamemode_toggle.disabled = False
    gamemode_toggle.value = 'Spieler vs Spieler'
    global game
    game = Game()
    clear_output(wait = True)
    game.update_canvas()

Im folgenden Programmcode werden, abhängig von den Variablen d und wdh, d*wdh-viele Spiele durchgerechnet und die die durchnittlichen Zugzeiten für die Algorithmen bestimmt. Hierzu wird nach jedem Spiel für Alha-Beta und Minimax die Gesamtzeit des jeweiligen Algorithmus durch die  Anzahl der Züge des Algorithmus geteilt. Somit entsteht die durschnittliche Zugzeit für den Algorithmus. 
* `max_depth`   ist die Tiefe, bis zu der Minimax mit Alpha-Beta verglichen werden soll
* `repetition` ist die Anzahl der Wiederholungen einer Tiefenberechnung die für die Bildunge des Durchschnitts genutzt werden

Bei mehrfachem Ausführen fällt hier auf, dass die Werte für d∈{1,2} stark schwanken und somit nicht besonders aussagekräftig sind. 

In [ ]:
import random 
import pandas as pd
%run ./Muehle_GUI.ipynb

#Variablen zur Konfiguration der Berechnung
max_depth  = 5 #Tiefe, bis zu der Verglichen werden soll
repetition = 3 #Anzahl der Wiederholungen einer Tiefe zur verbesserung des Ergebnisses

ab_avg      = []
mm_avg      = []
depth_array = []
ratio       = []

#Berechnung von depth=1 zu depth=d
for x in range(1,max_depth+1):
    #Um nicht nur einen Wert für eine Suchtiefe zu haben wird der Durchschnitt aus wdh-vielen Durchläufen berechnet
    ab_avg_      = []
    mm_avg_      = []
    for i in range(1,repetition+1):
        global game
        game = Game()
        clear_output(wait = True)
        game.ab_depth    = x
        game.mm_depth    = x
        gamemode_toggle.value = 'alpha-beta vs Minimax'
        game.pause = False
        game.update_canvas()
        game.algo_step()

        ab_avg_ += [game.ab_time/game.ab_moves]
        mm_avg_ += [game.mm_time/game.mm_moves]

        restart_auto_play()

    ab_avg += [sum(ab_avg_)/len(ab_avg_)]
    mm_avg += [sum(mm_avg_)/len(mm_avg_)]
    depth_array += [x]
    ratio += [mm_avg[-1]/ab_avg[-1]*100]

clear_output(wait = True)

#Plotting
df = pd.DataFrame(data={'Suchtiefe':depth_array,'Ø Zugzeit Alpha-Beta (s)':ab_avg, 'Ø Zugzeit Minimax (s)':mm_avg, 'MM/AB (%)':ratio})
df.index = df.index+1
df.plot(kind='line',x='Suchtiefe',y=["Ø Zugzeit Alpha-Beta (s)", "Ø Zugzeit Minimax (s)"])

Logarithmische Darstellung der Auswerung:

In [ ]:
df.plot(kind='line',x='Suchtiefe',y=["Ø Zugzeit Alpha-Beta (s)", "Ø Zugzeit Minimax (s)"],logy=True)

Tabellarische Darstellung der Auswerung:

In [ ]:
df

## Ausblick

Das entwickelte Mühle-Spiel ist in der Lage, verschiedene Spielmodi anzubieten. Das Spielen gegen eine KI, sowie das Spiel KI gegen KI ist möglich, sodass das grundlegende Ziel dieser Arbeit erfüllt wurde. Auch die Verwendung von Transpositionstabellen als Speicher für die bereits bewerteten Zustände wurde umgesetzt. Eine Bewertung der Ergebnisse wurde im vorangehenden Kapitel vorgenommen. Nun soll auf Möglichkeiten eingegangen werden, die die Performance optimieren oder die Stärke der KI steigern.

Die erste Möglichkeit hier ist die Verbesserung der Heuristik. Da die Heuristik die Bewertung der Zustände vornimmt und somit über den nächsten Zustand entscheidet, ist sie der Ansatzpunkt für Leistungssteigerungen. Aktuell umfasst die Heuristik die Aspekte Anzahl der Spielsteine sowie die Anzahl der geschlossenen Mühlen.  Überlegungen für weitere Aspekte sind die Betrachtung der Anzahl der Zugmöglichkeiten und Anzahl der offenen Mühlen sowie die Bewertung von bestimmten Stellungen. Kriterien für die Entscheidung sind die Veränderung der Rechenzeit und des Gewinnanteils. Ist Erhöhung der Rechenzeit möglichst niedrig, während der Anteil der Gewinne gesteigert werden kann, sollte der Aspekt in die Heuristik mit aufgenommen werden. Auch die Gewichtung der unterschiedlichen Aspekte ist ein Punkt, an dem noch eingegriffen und optimiert werden kann.

Eine weitere Verbesserungsmöglichkeit ist die Einführung einer Spieldatenbank zur Ermittlung besserer Züge. Prinzipiell ist so eine Datenbank für das gesamte Spiel nutzbar, aufgrund des hohen Speicherbedarfs wird häufig jedoch nur eine Endspieldatenbank umgesetzt. Gerade in der dritten Spielphase, wenn die Spieler viele Zugmöglichkeiten haben und der Spielbaum so stark verzweigt ist, kann dies hilfreich sein. Ein Grundstein für die Erstellung und Verwendung einer solchen Spieldatenbank wurde mit dem Abspeichern der Spiele schon gelegt. Eine richtige Verwendung der Datenbank lag jedoch nicht im Anforderungsrahmen dieser Studienarbeit. 

Eine weitere Optimierungsmöglichkeit, um schneller mehr Zustände abzudecken und den Speicherbedarf zu senken ist die Beachtung von Symmetrien und Drehungen. Die Grafik unten veranschaulicht die Symmetrien eines Spielfeldes. Die roten Achsen sind einfache Spiegelungen. Aber auch das Tauschen des inneren und äußeren Rings ist möglich. Dies wird durch die grünen Linien veranschaulicht.

<img src="images\symmetries.png" alt="Symmetrien eines Spielbrettes" width="300"/>

Auch eine Drehung des Spielfelds um 90, 180 oder 270 Grad verändert den eigentlichen Zustand nicht, weshalb auch hier eine Vereinfachung stattfinden kann.

Bei der Überlegung, ob Symmetrien und Drehungen betrachtet werden sollen, ist zu beachten, dass zwar der Speicheraufwand und die Anzahl der möglichen Zustände reduziert werden, die Rechenzeit jedoch stark zunimmt.

Abschließend ist zu sagen, dass noch weitere Verbesserungen möglich sind, dies aber immer unter Abwägung der Vor- und Nachteile und vor allem hinsichtlich der benötigten Rechenzeit geschehen sollte.